# Scraping Data BPS menggunakan Web API

Notebook ini digunakan untuk melakukan scraping data dari BPS (Badan Pusat Statistik) menggunakan Web API resmi mereka.

**Endpoint API:**
- Base URL: `https://webapi.bps.go.id/v1/api/list/model/data/lang/ind/domain/0000/var/2506`
- Variasi tahun: th/125 dan th/126

**Catatan:** Pastikan Anda sudah memiliki API Key dari BPS. Daftar di [https://webapi.bps.go.id](https://webapi.bps.go.id)

## 1. Import Library yang Diperlukan

In [1]:
import requests
import json
import pandas as pd
from datetime import datetime
import time

## 2. Konfigurasi API

In [2]:
# GANTI DENGAN API KEY ANDA DARI https://webapi.bps.go.id
API_KEY = '0a94470ebd2b059f522da5b53a491575'  # Ganti dengan API Key Anda

# Konfigurasi endpoint
BASE_URL = 'https://webapi.bps.go.id/v1/api/list/model/data/lang/ind/domain/0000/var/2506'

# Variasi tahun yang akan di-scrape
TAHUN_CODES = [125, 126]

# Fungsi untuk membuat URL endpoint
def buat_url(tahun_code):
    return f"{BASE_URL}/th/{tahun_code}/key/{API_KEY}"

# Tampilkan URL yang akan digunakan
for th in TAHUN_CODES:
    print(f"URL untuk th/{th}: {buat_url(th)}")

URL untuk th/125: https://webapi.bps.go.id/v1/api/list/model/data/lang/ind/domain/0000/var/2506/th/125/key/0a94470ebd2b059f522da5b53a491575
URL untuk th/126: https://webapi.bps.go.id/v1/api/list/model/data/lang/ind/domain/0000/var/2506/th/126/key/0a94470ebd2b059f522da5b53a491575


## 3. Fungsi untuk Scraping Data

In [3]:
def scrape_bps_data(tahun_code):
    """
    Fungsi untuk melakukan scraping data dari BPS WebAPI
    
    Parameters:
    - tahun_code: Kode tahun (125 atau 126)
    
    Returns:
    - Dictionary berisi response dari API
    """
    url = buat_url(tahun_code)
    
    try:
        print(f"\n{'='*60}")
        print(f"Mengambil data untuk tahun code: {tahun_code}")
        print(f"URL: {url}")
        print(f"{'='*60}")
        
        # Kirim request ke API
        response = requests.get(url, timeout=30)
        
        # Cek status code
        print(f"Status Code: {response.status_code}")
        
        if response.status_code == 200:
            data = response.json()
            print(f"✓ Berhasil mengambil data")
            
            # Tampilkan informasi dasar
            if 'data' in data:
                if isinstance(data['data'], list):
                    print(f"Jumlah data: {len(data['data'])} record")
                else:
                    print(f"Tipe data: {type(data['data'])}")
            
            return {
                'success': True,
                'tahun_code': tahun_code,
                'data': data,
                'timestamp': datetime.now().strftime('%Y-%m-%d %H:%M:%S')
            }
        else:
            print(f"✗ Error: Status code {response.status_code}")
            print(f"Response: {response.text[:200]}")
            
            return {
                'success': False,
                'tahun_code': tahun_code,
                'error': f"HTTP {response.status_code}",
                'message': response.text[:200],
                'timestamp': datetime.now().strftime('%Y-%m-%d %H:%M:%S')
            }
            
    except requests.exceptions.Timeout:
        print(f"✗ Error: Request timeout")
        return {
            'success': False,
            'tahun_code': tahun_code,
            'error': 'Timeout',
            'timestamp': datetime.now().strftime('%Y-%m-%d %H:%M:%S')
        }
        
    except requests.exceptions.RequestException as e:
        print(f"✗ Error: {str(e)}")
        return {
            'success': False,
            'tahun_code': tahun_code,
            'error': str(e),
            'timestamp': datetime.now().strftime('%Y-%m-%d %H:%M:%S')
        }

## 4. Jalankan Scraping untuk Semua Tahun

In [4]:
# Dictionary untuk menyimpan semua hasil
hasil_scraping = {}

# Loop untuk setiap tahun
for tahun_code in TAHUN_CODES:
    hasil = scrape_bps_data(tahun_code)
    hasil_scraping[f'th_{tahun_code}'] = hasil
    
    # Delay sebentar antara request untuk menghindari rate limiting
    if tahun_code != TAHUN_CODES[-1]:
        print("\nMenunggu 2 detik sebelum request berikutnya...")
        time.sleep(2)

print("\n" + "="*60)
print("SCRAPING SELESAI")
print("="*60)


Mengambil data untuk tahun code: 125
URL: https://webapi.bps.go.id/v1/api/list/model/data/lang/ind/domain/0000/var/2506/th/125/key/0a94470ebd2b059f522da5b53a491575
Status Code: 200
✓ Berhasil mengambil data

Menunggu 2 detik sebelum request berikutnya...

Mengambil data untuk tahun code: 126
URL: https://webapi.bps.go.id/v1/api/list/model/data/lang/ind/domain/0000/var/2506/th/126/key/0a94470ebd2b059f522da5b53a491575
Status Code: 200
✓ Berhasil mengambil data

SCRAPING SELESAI


## 5. Tampilkan Ringkasan Hasil

In [5]:
# Tampilkan ringkasan hasil
print("\nRINGKASAN HASIL SCRAPING:")
print("="*60)

for key, result in hasil_scraping.items():
    print(f"\n{key}:")
    print(f"  Status: {'✓ Berhasil' if result['success'] else '✗ Gagal'}")
    print(f"  Timestamp: {result['timestamp']}")
    
    if result['success']:
        if 'data' in result and result['data']:
            # Tampilkan keys yang ada di response
            print(f"  Response keys: {list(result['data'].keys())}")
            
            # Jika ada field 'data', tampilkan info lebih detail
            if 'data' in result['data']:
                data_content = result['data']['data']
                if isinstance(data_content, list) and len(data_content) > 0:
                    print(f"  Jumlah record: {len(data_content)}")
                    print(f"  Sample data pertama: {data_content[0] if data_content else 'Kosong'}")
    else:
        print(f"  Error: {result.get('error', 'Unknown error')}")


RINGKASAN HASIL SCRAPING:

th_125:
  Status: ✓ Berhasil
  Timestamp: 2026-02-05 08:31:31
  Response keys: ['status', 'data-availability', 'last_update', 'subject', 'var', 'turvar', 'labelvervar', 'vervar', 'tahun', 'turtahun', 'datacontent', 'related']

th_126:
  Status: ✓ Berhasil
  Timestamp: 2026-02-05 08:31:33
  Response keys: ['status', 'data-availability', 'last_update', 'subject', 'var', 'turvar', 'labelvervar', 'vervar', 'tahun', 'turtahun', 'datacontent', 'related']


## 6. Eksplorasi Data (untuk data yang berhasil di-scrape)

In [6]:
# Cek data yang berhasil di-scrape
for key, result in hasil_scraping.items():
    if result['success'] and 'data' in result:
        print(f"\n{'='*60}")
        print(f"DETAIL DATA: {key}")
        print(f"{'='*60}")
        
        # Tampilkan struktur data lengkap (pretty print)
        print(json.dumps(result['data'], indent=2, ensure_ascii=False)[:2000])  # Batasi 2000 karakter
        print("\n... (data dipotong untuk tampilan)")
        
        break  # Hanya tampilkan satu contoh


DETAIL DATA: th_125
{
  "status": "OK",
  "data-availability": "available",
  "last_update": "2026-02-02 13:09:27",
  "subject": [
    {
      "val": 557,
      "label": "Pertanian, Kehutanan, Perikanan"
    }
  ],
  "var": [
    {
      "val": 2506,
      "label": "Produksi Padi Menurut Provinsi (Bulanan)",
      "unit": "Ton",
      "subj": "Tanaman Pangan",
      "def": "",
      "decimal": 2,
      "note": "1. Produksi padi Januari-Maret adalah angka potensi\n2. Kualitas produksi adalah Gabah Kering Giling (GKG)\n3. Data pokok tanaman pangan yang dikumpulkan oleh Badan Pusat Statistik adalah luas panen dan produktivitas (hasil per hektar). Produksi merupakan hasil perkalian antara luas panen dan produktivitas."
    }
  ],
  "turvar": [
    {
      "val": "0",
      "label": "Tidak ada"
    }
  ],
  "labelvervar": "38 Provinsi",
  "vervar": [
    {
      "val": 1100,
      "label": "ACEH"
    },
    {
      "val": 1200,
      "label": "SUMATERA UTARA"
    },
    {
      "val": 1300

## 7. Konversi ke DataFrame (jika data berbentuk list)

In [7]:
# Dictionary untuk menyimpan DataFrame
dataframes = {}

for key, result in hasil_scraping.items():
    if result['success'] and 'data' in result:
        response_data = result['data']
        
        # Coba konversi ke DataFrame jika data berbentuk list
        if 'data' in response_data:
            data_list = response_data['data']
            
            if isinstance(data_list, list) and len(data_list) > 0:
                try:
                    df = pd.DataFrame(data_list)
                    dataframes[key] = df
                    
                    print(f"\n{key}: Berhasil dikonversi ke DataFrame")
                    print(f"Shape: {df.shape}")
                    print(f"Columns: {list(df.columns)}")
                    print(f"\nSample data (5 baris pertama):")
                    print(df.head())
                    
                except Exception as e:
                    print(f"\n{key}: Gagal konversi ke DataFrame - {str(e)}")
            else:
                print(f"\n{key}: Data bukan list atau kosong")
        else:
            print(f"\n{key}: Tidak ada field 'data' dalam response")


th_125: Tidak ada field 'data' dalam response

th_126: Tidak ada field 'data' dalam response


## 8. Simpan Data ke File JSON

In [8]:
# Simpan setiap hasil ke file JSON terpisah
for key, result in hasil_scraping.items():
    if result['success'] and 'data' in result:
        filename = f"bps-data-var2506-{key}.json"
        
        with open(filename, 'w', encoding='utf-8') as f:
            json.dump(result['data'], f, ensure_ascii=False, indent=2)
        
        print(f"✓ Data {key} berhasil disimpan ke: {filename}")

# Simpan semua hasil (termasuk metadata) ke satu file
filename_all = f"bps-scraping-results-{datetime.now().strftime('%Y%m%d_%H%M%S')}.json"
with open(filename_all, 'w', encoding='utf-8') as f:
    json.dump(hasil_scraping, f, ensure_ascii=False, indent=2)

print(f"\n✓ Semua hasil scraping disimpan ke: {filename_all}")

✓ Data th_125 berhasil disimpan ke: bps-data-var2506-th_125.json
✓ Data th_126 berhasil disimpan ke: bps-data-var2506-th_126.json

✓ Semua hasil scraping disimpan ke: bps-scraping-results-20260205_083134.json


## 9. Simpan DataFrame ke CSV (jika ada)

In [9]:
# Simpan DataFrame ke CSV jika ada
if dataframes:
    for key, df in dataframes.items():
        filename = f"bps-data-var2506-{key}.csv"
        df.to_csv(filename, index=False, encoding='utf-8-sig')
        print(f"✓ DataFrame {key} berhasil disimpan ke: {filename}")
else:
    print("Tidak ada DataFrame yang berhasil dibuat")

Tidak ada DataFrame yang berhasil dibuat


## 10. Analisis Tambahan (Optional)

Gunakan cell di bawah ini untuk melakukan analisis lebih lanjut terhadap data yang telah di-scrape.

In [10]:
# Contoh: Bandingkan data antara th/125 dan th/126
if len(dataframes) >= 2:
    print("Perbandingan data antar tahun:")
    print("="*60)
    
    for key, df in dataframes.items():
        print(f"\n{key}:")
        print(f"  Jumlah baris: {len(df)}")
        print(f"  Jumlah kolom: {len(df.columns)}")
        if not df.empty:
            print(f"  Info DataFrame:")
            print(df.info())
else:
    print("# Tulis kode analisis Anda di sini")
    print("# Contoh: df.describe(), df.groupby(), dll.")

# Tulis kode analisis Anda di sini
# Contoh: df.describe(), df.groupby(), dll.
